In [ ]:
!pip install vaderSentiment

     |████████████████████████████████| 125 kB 7.3 MB/s 


In [ ]:
! pip install PyDrive

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import re

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1Cus0W0LLYX7nhl6XLV50iINRh72cRCEv/view?usp=sharing'

In [ ]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Survey_Results_Labeled_Feb.csv') 
 
df = pd.read_csv('Survey_Results_Labeled_Feb.csv')
df


,Unnamed: 0,text,1,2,3,Label
0,Q1,"""Human rights groups have raised alarm over wh...",0.0,0.7,0.3,neutral
1,Q2,@KingJames You have been awfully and painfully...,0.0,0.0,1.0,negative
2,Q3,"""@PrillLee @MarthaP29033882 @DanRather It is n...",0.0,0.3,0.7,negative
3,Q4,@AntoineSallesP @JoshuaHabib4 @JLMelenchon Why...,0.3,0.5,0.2,neutral
4,Q5,"@IAmMrKobayashi World War III; Full video, How...",0.2,0.7,0.1,neutral
...,...,...,...,...,...,...
97,Q96,There are 7 or 8 Billionaires in the UKRAINE! ...,0.4,0.0,0.6,negative
98,Q97,"""@EP_President @Europarl_EN @EU_Commission STO...",0.0,0.3,0.7,negative
99,Q98,"""@vonderleyen STOP RUSSIAN AGGRESSION AGAINST ...",0.0,0.1,0.9,negative
100,Q99,"""""However difficult it may be, our pain will b...",0.4,0.1,0.5,negative


In [ ]:
df['text'] = df['text'].astype('str') 

In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt
def clean_tweets(tweets):
    #remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:") 
    
    #remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
    #remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    
    #remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    
    return tweets

In [ ]:
df['text'] = clean_tweets(df['text'])

In [ ]:
df['text'].head()

0    "Human rights groups have raised alarm over wh...
1     You have been awfully and painfully quiet abo...
2    "   It is nothing like California rising up ag...
3       Why the Ukraine Crisis Is the West’s Fault"...
4     World War III; Full video, How Russia Begins ...
Name: text, dtype: object

In [ ]:
scores = []

# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []

for i in range(df['text'].shape[0]):
    #print(analyser.polarity_scores(sentiments_pd['text'][i]))
    compound = analyser.polarity_scores(df['text'][i])["compound"]
    pos = analyser.polarity_scores(df['text'][i])["pos"]
    neu = analyser.polarity_scores(df['text'][i])["neu"]
    neg = analyser.polarity_scores(df['text'][i])["neg"]
    
    scores.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg,
                       "Neutral": neu
                  })

In [ ]:
df

,Unnamed: 0,text,1,2,3,Label
0,Q1,"""Human rights groups have raised alarm over wh...",0.0,0.7,0.3,neutral
1,Q2,You have been awfully and painfully quiet abo...,0.0,0.0,1.0,negative
2,Q3,""" It is nothing like California rising up ag...",0.0,0.3,0.7,negative
3,Q4,"Why the Ukraine Crisis Is the West’s Fault""...",0.3,0.5,0.2,neutral
4,Q5,"World War III; Full video, How Russia Begins ...",0.2,0.7,0.1,neutral
...,...,...,...,...,...,...
97,Q96,There are 7 or 8 Billionaires in the UKRAINE! ...,0.4,0.0,0.6,negative
98,Q97,""" STOP RUSSIAN AGGRESSION AGAINST #UKRAINE. ...",0.0,0.3,0.7,negative
99,Q98,""" STOP RUSSIAN AGGRESSION AGAINST #UKRAINE. C...",0.0,0.1,0.9,negative
100,Q99,"""""However difficult it may be, our pain will b...",0.4,0.1,0.5,negative


In [ ]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score)

In [ ]:
df.head(10)

,Unnamed: 0,text,1,2,3,Label,Compound,Positive,Negative,Neutral
0,Q1,"""Human rights groups have raised alarm over wh...",0.0,0.7,0.3,neutral,-0.8519,0.000,0.231,0.769
1,Q2,You have been awfully and painfully quiet abo...,0.0,0.0,1.0,negative,-0.3268,0.000,0.075,0.925
2,Q3,""" It is nothing like California rising up ag...",0.0,0.3,0.7,negative,-0.6020,0.046,0.138,0.816
3,Q4,"Why the Ukraine Crisis Is the West’s Fault""...",0.3,0.5,0.2,neutral,-0.9004,0.000,0.286,0.714
4,Q5,"World War III; Full video, How Russia Begins ...",0.2,0.7,0.1,neutral,-0.6360,0.000,0.244,0.756
5,Q6,"#sauce?? - with friends like China and India,...",0.2,0.1,0.7,negative,0.2755,0.223,0.189,0.588
6,Q7,No doubt he would be ordering US troops to fi...,0.1,0.6,0.3,neutral,-0.1255,0.126,0.156,0.718
7,Q8,Joe Biden did that 👉 Dow futures tumble 260 p...,0.0,0.5,0.5,neutral,0.0000,0.000,0.000,1.000
8,Q9,"""Hats off Ukraine. 🥺\n#Ukraine #Russia #Volody...",0.7,0.1,0.2,positive,0.0000,0.000,0.000,1.000
9,Q10,"""#IStandWithUkraine\n\nUkraine's ambassador to...",0.3,0.4,0.3,neutral,-0.2023,0.084,0.116,0.800


In [ ]:
score = df["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05 :
        sentiment.append('positive')
    elif i <= -0.05 :
        sentiment.append('negative')
    else:
        sentiment.append('neutral')
df["Sentiment"] = sentiment
df.head()

,Unnamed: 0,text,1,2,3,Label,Compound,Positive,Negative,Neutral,Sentiment
0,Q1,"""Human rights groups have raised alarm over wh...",0.0,0.7,0.3,neutral,-0.8519,0.000,0.231,0.769,negative
1,Q2,You have been awfully and painfully quiet abo...,0.0,0.0,1.0,negative,-0.3268,0.000,0.075,0.925,negative
2,Q3,""" It is nothing like California rising up ag...",0.0,0.3,0.7,negative,-0.6020,0.046,0.138,0.816,negative
3,Q4,"Why the Ukraine Crisis Is the West’s Fault""...",0.3,0.5,0.2,neutral,-0.9004,0.000,0.286,0.714,negative
4,Q5,"World War III; Full video, How Russia Begins ...",0.2,0.7,0.1,neutral,-0.6360,0.000,0.244,0.756,negative


In [ ]:
#delete control tweets from set
df = df.drop(33)
df = df.drop(67)

In [ ]:
#change column names to make clear what is what
df = df.rename(columns={"Label":"Manual Label"})
df = df.rename(columns={"Sentiment":"Vader Label"})

In [ ]:
df

,Unnamed: 0,text,1,2,3,Manual Label,Compound,Positive,Negative,Neutral,Vader Label
0,Q1,"""Human rights groups have raised alarm over wh...",0.0,0.7,0.3,neutral,-0.8519,0.000,0.231,0.769,negative
1,Q2,You have been awfully and painfully quiet abo...,0.0,0.0,1.0,negative,-0.3268,0.000,0.075,0.925,negative
2,Q3,""" It is nothing like California rising up ag...",0.0,0.3,0.7,negative,-0.6020,0.046,0.138,0.816,negative
3,Q4,"Why the Ukraine Crisis Is the West’s Fault""...",0.3,0.5,0.2,neutral,-0.9004,0.000,0.286,0.714,negative
4,Q5,"World War III; Full video, How Russia Begins ...",0.2,0.7,0.1,neutral,-0.6360,0.000,0.244,0.756,negative
...,...,...,...,...,...,...,...,...,...,...,...
97,Q96,There are 7 or 8 Billionaires in the UKRAINE! ...,0.4,0.0,0.6,negative,-0.2244,0.054,0.077,0.870,negative
98,Q97,""" STOP RUSSIAN AGGRESSION AGAINST #UKRAINE. ...",0.0,0.3,0.7,negative,-0.7140,0.115,0.334,0.551,negative
99,Q98,""" STOP RUSSIAN AGGRESSION AGAINST #UKRAINE. C...",0.0,0.1,0.9,negative,-0.7140,0.115,0.334,0.551,negative
100,Q99,"""""However difficult it may be, our pain will b...",0.4,0.1,0.5,negative,-0.8658,0.000,0.199,0.801,negative


In [ ]:
 import sklearn
 from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support
 b_a_Vader = balanced_accuracy_score(df['Manual Label'], df['Vader Label'])
 p_r_f1_class_Vader = precision_recall_fscore_support(df['Manual Label'], df['Vader Label'], average=None, labels=['negative', 'neutral', 'positive'])
 p_r_f1_Vader = precision_recall_fscore_support(df['Manual Label'], df['Vader Label'], average='macro')

In [ ]:
df['Manual Label'].value_counts()

negative    46
neutral     44
positive    10
Name: Manual Label, dtype: int64

In [ ]:
#0.4 accuracy with pre-processing
print(b_a_Vader)
print(p_r_f1_Vader)
print(p_r_f1_class_Vader)

0.42127799736495386
(0.4, 0.4, 0.4000000000000001, None)
(array([0.49230769, 0.42857143, 0.17857143]), array([0.69565217, 0.06818182, 0.5       ]), array([0.57657658, 0.11764706, 0.26315789]), array([46, 44, 10]))


In [ ]:
import pandas as pd
y_actu = df['Manual Label']
y_pred = df['Vader Label']
df_confusion = pd.crosstab(y_actu, y_pred)

In [ ]:
df_confusion

Vader Label,negative,neutral,positive
Manual Label,,,
negative,32,1,13
neutral,31,3,10
positive,2,3,5


In [ ]:
#Get accuracies per class order (negative, neutral, positive)
from sklearn.metrics import confusion_matrix
y_true = df['Manual Label']
y_pred = df['Vader Label']
matrix = confusion_matrix(y_true, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

array([0.69565217, 0.06818182, 0.5       ])

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
df.to_csv('VADER_Test_Feb.csv')
!cp VADER_Test_Feb.csv "/content/drive/My Drive/Colab Notebooks"

